In [33]:
import numpy as np
import simpy
from collections import OrderedDict
from collections import Counter

processList = list()

class AMIDOLRateReward():
    def __init__(self):
        self.rewards = dict()
    
    def accumulateReward(self, env, params):
        return(0.0)
    
    def getDelay(self, params):
        return(simpy.core.Infinity)
    
    def isEnabled(self, params):
        return(True)
    
    def simpyProcess(self, env, params):
        while(True):
            try:
                if(self.isEnabled(params)):
                    yield env.timeout(self.getDelay(params))
                    self.accumulateReward(env, params)
                else:
                    yield env.timeout(simpy.core.Infinity)
            except simpy.Interrupt as i:
                continue
        print(self.getName() + " terminating.")
        
class AMIDOLEvent():
    def getName(self):
        return("GenericEvent")
    
    def getRate(self, params):
        return(1.0)
    
    def getDelay(self, params):
        delay = np.random.exponential(self.getRate(params))
        return(delay)
    
    def isEnabled(self, params):
        return(True)
    
    def fireEvent(self, params):
        return(params)
    
    def reactivation(self, env):
        global processList
        for process in processList:
            if (process != env.active_process):
                process.interrupt()
    
    def simpyProcess(self, env, params):
        while(True):
            try:
                if(self.isEnabled(params)):
                    yield env.timeout(self.getDelay(params))
                    self.fireEvent(params)
                else:
                    yield env.timeout(simpy.core.Infinity)
                self.reactivation(env)
            except simpy.Interrupt as i:
                continue
        print(self.getName() + " terminating.")

In [38]:
class AMIDOLParameters():
    def __init__(self):
        self.S_Pcinfect_S = 51999999
        self.ScinfectcI_Scinfect_IcI_Pccure_I = 1
        self.ScinfectcIccure_RcR_P = 0
        self.beta = 1.0/3.0*1.24
        self.gamma = 1.0/3.0
        
class infectEvent(AMIDOLEvent):
    def getName(self):
        return("InfectEvent")
    
    def getRate(self, v):
        rate = 1.0 / (v.beta*v.S_Pcinfect_S*v.ScinfectcI_Scinfect_IcI_Pccure_I/(v.S_Pcinfect_S+v.ScinfectcI_Scinfect_IcI_Pccure_I+v.ScinfectcIccure_RcR_P))
        return(rate)
    
    def isEnabled(self, v):
        return((v.beta*v.S_Pcinfect_S * v.ScinfectcI_Scinfect_IcI_Pccure_I) > 0.0)
    
    def fireEvent(self, v):
        v.S_Pcinfect_S -= 1.0
        v.ScinfectcI_Scinfect_IcI_Pccure_I += 1.0
        
class cureEvent(AMIDOLEvent):
    
    def getName(self):
        return("CureEvent")
    
    def getRate(self, v):
        rate = 1.0/(v.gamma*v.ScinfectcI_Scinfect_IcI_Pccure_I)
        return(rate)
    
    def isEnabled(self, v):
        return(v.gamma*v.ScinfectcI_Scinfect_IcI_Pccure_I > 0.0)
    
    def fireEvent(self, v):
        v.ScinfectcI_Scinfect_IcI_Pccure_I -= 1.0
        v.ScinfectcIccure_RcR_P += 1.0
        
class rvIRateReward(AMIDOLRateReward):
    def __init__(self):
        self.rewards = OrderedDict()
        self.samplePoints = [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0]
        self.delays = list()
        self.delays.append(self.samplePoints[0])
        idx = 1
        lastX = self.samplePoints[0]
        for x in self.samplePoints[1:]:
            self.delays.append(x - lastX)
            lastX = x
    
    def accumulateReward(self, env, params):
        self.rewards[env.now] = params.ScinfectcI_Scinfect_IcI_Pccure_I
        
    
    def getDelay(self, params):
        if (self.delays):
            return(self.delays.pop(0))
        else:
            return(simpy.core.Infinity)
        
rvICounter = Counter()
maxRuns = 100

for trace in range(0, maxRuns):
    params = AMIDOLParameters()
    cure = cureEvent()
    infect = infectEvent()
    rvI = rvIRateReward()

    env = simpy.Environment()
    processList = []
    cureProcess = env.process(cure.simpyProcess(env, params))
    processList.append(cureProcess)
    infectProcess = env.process(infect.simpyProcess(env, params))
    processList.append(infectProcess)
    rvIProcess = env.process(rvI.simpyProcess(env, params))

    env.run(until=rvI.samplePoints[-1])
    results = {k: v / maxRuns for k, v in rvI.rewards.iteritems()}
    rvICounter = Counter(results) + rvICounter
    
rvICounter

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Counter({5.0: 1.7900000000000007,
         10.0: 2.1700000000000004,
         15.0: 3.3400000000000007,
         20.0: 5.049999999999999,
         25.0: 8.85,
         30.0: 12.309999999999999,
         35.0: 16.860000000000003,
         40.0: 27.09,
         45.0: 38.48,
         50.0: 58.47999999999999,
         55.0: 84.88000000000001,
         60.0: 125.53999999999996,
         65.0: 186.67000000000002,
         70.0: 275.11999999999995,
         75.0: 417.18999999999994,
         80.0: 632.41,
         85.0: 946.8800000000001,
         90.0: 1423.2700000000002,
         95.0: 2126.1999999999994})

In [14]:
counter = Counter()
for trace in range(0, 100):
    env.run(until=rvI.samplePoints[-1])
    results = 

{0.0: 1,
 5.0: 1,
 10.0: 1.0,
 15.0: 2.0,
 20.0: 20.0,
 25.0: 38.0,
 30.0: 66.0,
 35.0: 99.0,
 40.0: 108.0,
 45.0: 145.0,
 50.0: 281.0,
 55.0: 486.0,
 60.0: 719.0,
 65.0: 1002.0,
 70.0: 1459.0,
 75.0: 2205.0,
 80.0: 3206.0,
 85.0: 4631.0}